- CMAKE_ARGS="-DGGML_CUDA=on" pip install llama-cpp-python

In [1]:
# !CMAKE_ARGS="-DGGML_CUDA=on" FORCE_CMAKE=1 pip install --upgrade --force-reinstall llama-cpp-python --no-cache-dir

In [2]:
# !CMAKE_ARGS="-DGGML_CUDA=on" FORCE_CMAKE=1 pip install --no-cache-dir llama-cpp-python==0.3.12 --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu124

In [3]:
# !CMAKE_ARGS="-DGGML_CUDA=on" FORCE_CMAKE=1 pip install --upgrade --force-reinstall llama-cpp-python==0.3.4 --no-cache-dir --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu124

In [4]:
# %pip install --no-cache-dir llama-cpp-python==0.3.14 --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu124
# %pip install --no-cache-dir llama-cpp-python==0.3.13 --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu124
# %pip install --no-cache-dir llama-cpp-python==0.3.12 --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu124

In [5]:
# 첫 번째 셀에서 실행
import os
import sys

# 환경 변수 설정
# os.environ["GGML_CUDA"] = "1"
# os.environ["LLAMA_CUBLAS"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

# llama-cpp-python 재설치 (환경 변수 적용)
# import subprocess
# subprocess.run([
#     "pip", "install", "llama-cpp-python", 
#     "--force-reinstall", "--no-cache-dir"
# ], check=True)

In [6]:
import torch
import pandas as pd
from pathlib import Path
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFDirectoryLoader, CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

import multiprocessing
from langchain_community.chat_models import ChatLlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# RAG 체인 구성
from langchain.chains import RetrievalQA



print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print((torch.cuda._get_nvml_device_index(0)))

True
NVIDIA A100-SXM4-80GB
3


In [7]:
# Path 객체에는 -r 옵션(재귀적 검색)이 없습니다.
# 만약 특정 디렉토리 내에서 재귀적으로 파일을 찾고 싶다면 glob 메서드나 rglob 메서드를 사용해야 합니다.
# 아래는 예시입니다.

root_path = Path(".").resolve().parent.parent
model_path = root_path / "ai_models" / "base_models" / "BGE-m3-ko"
print(model_path)

/AIHUB/PCNRND/home/chatbot/ai_models/base_models/BGE-m3-ko


# 

In [8]:
root_path

PosixPath('/AIHUB/PCNRND/home/chatbot')

In [9]:
# 임베딩 모델 초기화
embeddings = HuggingFaceEmbeddings(
    model_name=str(model_path),
    model_kwargs={'device': 'cuda:0'},
    encode_kwargs={'normalize_embeddings': True}
)

/tmp/ipykernel_554817/3262055005.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [10]:
faiss_index_directory = "./data/faiss_csv_db"
vectorstore = FAISS.load_local(faiss_index_directory, embeddings, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(    
    search_type="similarity_score_threshold", 
    search_kwargs={"score_threshold": 0.5, "k": 3}
)

In [11]:
# /AIHUB/PCNRND/home/chatbot/ai_models/gguf_models/llama-3.2-Korean-Bllossom-3B-gguf-Q4_K_M/llama-3.2-Korean-Bllossom-3B-gguf-Q4_K_M.gguf
# /AIHUB/PCNRND/home/chatbot/ai_models/gguf_models/MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M/llama-3-Korean-Bllossom-8B-Q4_K_M.gguf
# llm_path = "/ai_models/gguf_models/EXAONE-3.5-7.8B-Instruct-GGUF/EXAONE-3.5-7.8B-Instruct-Q4_K_M.gguf"
# print(f"LLM path: {llm_path}")

In [12]:
# /AIHUB/PCNRND/home/chatbot/ai_models/gguf_models/MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M/llama-3-Korean-Bllossom-8B-Q4_K_M.gguf
# /AIHUB/PCNRND/home/chatbot/ai_models/gguf_models/Bllossom/llama-3.2-Korean-Bllossom-3B-gguf-Q4_K_M/llama-3.2-Korean-Bllossom-3B-gguf-Q4_K_M.gguf

In [13]:
# llm_path = str(root_path / "ai_models" / "gguf_models" / "MLP-KTLim" / "llama-3-Korean-Bllossom-8B-gguf-Q4_K_M" / "llama-3-Korean-Bllossom-8B-Q4_K_M.gguf")
# llm_path = str(root_path / "ai_models" / "gguf_models" / "Bllossom" / "llama-3.2-Korean-Bllossom-3B-gguf-Q4_K_M" / "llama-3.2-Korean-Bllossom-3B-gguf-Q4_K_M.gguf")
llm_path = str(root_path / "ai_models" / "gguf_models" / "Bllossom" / "llama-3.2-Korean-Bllossom-3B-gguf-Q4_K_M" / "llama-3.2-Korean-Bllossom-3B-gguf-Q4_K_M.gguf")

In [14]:
# llm_path = str(root_path / "ai_models" / "base_models" / "llama-3.2-Korean-Bllossom-3B-gguf-Q4_K_M" / "llama-3.2-Korean-Bllossom-3B-gguf-Q4_K_M.gguf")

# print(llm_path)

In [15]:
llm_path

'/AIHUB/PCNRND/home/chatbot/ai_models/gguf_models/Bllossom/llama-3.2-Korean-Bllossom-3B-gguf-Q4_K_M/llama-3.2-Korean-Bllossom-3B-gguf-Q4_K_M.gguf'

In [16]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

llm = ChatLlamaCpp(
    model_path=llm_path,
    backend="cuda",      # ← 이 옵션을 반드시 추가하세요
    n_gpu_layers=-1,     # 전체 28개 레이어를 GPU에 할당
    n_ctx=8192,
    n_batch=64,
    temperature=0.1,
    max_tokens=512,
    top_p=0.9,
    repeat_penalty=1.1,
    use_mlock=True,
    use_mmap=True,
    callback_manager=callback_manager,
    n_threads=multiprocessing.cpu_count() - 1,
    # verbose=True,     # 할당 로그를 보고 싶다면 True
)

llama_model_loader: loaded meta data with 33 key-value pairs and 255 tensors from /AIHUB/PCNRND/home/chatbot/ai_models/gguf_models/Bllossom/llama-3.2-Korean-Bllossom-3B-gguf-Q4_K_M/llama-3.2-Korean-Bllossom-3B-gguf-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3.2 Korean Bllossom 3B
llama_model_loader: - kv   3:                           general.basename str              = llama-3.2-Korean-Bllossom
llama_model_loader: - kv   4:                         general.size_label str              = 3B
llama_model_loader: - kv   5:                            general.license str              = llama3.2
llama_model_loade

In [17]:
# 프롬프트 구성
question = "피씨엔 연구과제에 대해 알려줘"
prompt = f"<|im_start|>system\n당신은 도움이 되는 AI 어시스턴트입니다. 질문에 대해 3문장 정도 정확하고 유용한 답변을 제공해주세요.<|im_end|>\n<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant\n"

In [18]:
from sentence_transformers import CrossEncoder

In [19]:
# 2) 1차 검색기로부터 top_k 문서 가져오기
top_k = 20
retrieved_docs = vectorstore.similarity_search(question, k=top_k)

In [20]:
# 3) CrossEncoder 기반 Re-ranker 준비
reranker = CrossEncoder(model_path)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at /AIHUB/PCNRND/home/chatbot/ai_models/base_models/BGE-m3-ko and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
# 4) (query, passage) 쌍으로 점수 계산
pairs = [(question, doc.page_content) for doc in retrieved_docs]
scores = reranker.predict(pairs)

# 5) 점수를 기준으로 정렬
scored_docs = list(zip(retrieved_docs, scores))
scored_docs.sort(key=lambda x: x[1], reverse=True)

In [22]:
# 6) 상위 N개 선택
top_n = 5
final_docs = [doc for doc, score in scored_docs[:top_n]]

In [23]:
final_docs

[Document(id='42cac6c0-14a1-4c53-8156-1824359c3682', metadata={'source': './PCN_지식재산권현황.csv', 'row': 30}, page_content='출원: 등록\n연구개발기관명(소유권자): ㈜피씨엔\n지식재산권명: 엣지클라우드기반의물류네트워크관제시스템\n국가명: 대한민국\n출원ㆍ등록번호/출원ㆍ등록일: 10-2591748호/2023-10-17\n비고: 추가\n상태: 등록확인'),
 Document(id='227729ff-1e84-45b6-b57c-241d3d6db120', metadata={'source': './PCN_지식재산권현황.csv', 'row': 14}, page_content='출원: 등록\n연구개발기관명(소유권자): ㈜피씨엔\n지식재산권명: 오픈API 기반 콘텐츠 서비스 인터페이스 제공시스템 및 방법\n국가명: 대한민국\n출원ㆍ등록번호/출원ㆍ등록일: 10-1296321/2013-08-07\n비고: \n상태: 등록확인'),
 Document(id='4c78bec4-60fa-42de-a78b-3cbce66856af', metadata={'source': './PCN_지식재산권현황.csv', 'row': 3}, page_content='출원: 출원\n연구개발기관명(소유권자): ㈜피씨엔\n지식재산권명: 멀티 클라우드 환경에서의 클라우드 네이티브 기반 실시간 XR 클라우드 컴퓨팅 시스템\n국가명: 대한민국\n출원ㆍ등록번호/출원ㆍ등록일: 10-2024-0187440/ 2024-12-16\n비고: \n상태: 진행중'),
 Document(id='52fa1e61-5620-476b-be2a-fd21a6bda042', metadata={'source': './PCN_지식재산권현황.csv', 'row': 8}, page_content='출원: 출원\n연구개발기관명(소유권자): ㈜피씨엔\n지식재산권명: 빅데이터 신뢰성과 활용성 극대화를 위한 빅데이터 증강분석 프로파일링 방법 및 장치\n국가명: 대한민국\

In [24]:
retriever = vectorstore.as_retriever(search_kwargs={"k": top_n})
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,  # 로컬 LLM이면 교체
    retriever=retriever,
    return_source_documents=True
)


In [ ]:
context = "\n\n".join([doc.page_content for doc in final_docs])
prompt = f"다음 문서를 참고하여 질문에 답변해줘:\n\n{context}\n\n질문: {question}"
result = qa_chain.invoke(prompt)